In [5]:
pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218921 sha256=8e479c9194c5873bc71fe23d3c042ba88550046c6e27f3c217edd44fb271caa8
  Stored in directory: /root/.cache/pip/wheels/1e/5c/47/759b32beb377aee150eb4c1f1b2b7635032cc19ea09ce076fd
Successfully built pandas_ta


In [32]:
import pandas as pd
import yfinance as yf

dfSPY=yf.download("AMZN",start='2011-01-05', end='2021-01-05')

dfSPY=dfSPY[dfSPY.High!=dfSPY.Low]
dfSPY.reset_index(inplace=True)
dfSPY.head()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
0,2011-01-05,9.205,9.3725,9.2035,9.3710,9.3710,68376000
1,2011-01-06,9.325,9.3705,9.2625,9.2930,9.2930,63594000
2,2011-01-07,9.394,9.4225,9.1870,9.2745,9.2745,104434000
3,2011-01-10,9.252,9.2645,9.1255,9.2340,9.2340,67518000
4,2011-01-11,9.271,9.3000,9.1605,9.2170,9.2170,56284000


In [33]:
import pandas_ta as ta
dfSPY['EMA']=ta.ema(dfSPY.Close, length=200)
dfSPY['RSI']=ta.rsi(dfSPY.Close, length=2)
a=ta.adx(dfSPY.High, dfSPY.Low, dfSPY.Close, length=14)
dfSPY=dfSPY.join(a.ADX_14)

dfSPY.dropna(inplace=True)
dfSPY.reset_index(inplace=True)
dfSPY[420:425]

,index,Date,Open,High,Low,Close,Adj Close,Volume,EMA,RSI,ADX_14
420,619,2013-06-24,13.5645,13.6580,13.2500,13.5305,13.5305,77276000,12.789358,8.200302,18.848699
421,620,2013-06-25,13.6150,13.6735,13.4500,13.6045,13.6045,48992000,12.797469,42.893522,17.930779
422,621,2013-06-26,13.6990,13.8990,13.6750,13.8785,13.8785,60590000,12.808225,84.966827,17.125734
423,622,2013-06-27,13.9590,14.0100,13.7270,13.8775,13.8775,50390000,12.818865,84.512158,16.775119
424,623,2013-06-28,13.8095,13.9915,13.8095,13.8845,13.8845,63866000,12.829468,85.591167,16.449547


In [34]:
def addemasignal(df, backcandles):
    emasignal = [0]*len(df)
    for row in range(backcandles, len(df)):
        upt = 1
        dnt = 1
        for i in range(row-backcandles, row+1):
            if df.High[i]>=df.EMA[i]:
                dnt=0
            if df.Low[i]<=df.EMA[i]:
                upt=0
        if upt==1 and dnt==1:
            #print("!!!!! check trend loop !!!!")
            emasignal[row]=3
        elif upt==1:
            emasignal[row]=2
        elif dnt==1:
            emasignal[row]=1
    df['EMASignal'] = emasignal

addemasignal(dfSPY,6)

In [35]:
def totalSignal(df, ADX):
    ordersignal=[0]*len(df)
    for i in range(0, len(df)):
        if  df.RSI[i]<=25 and ADX[i]>=20 and df.EMASignal[i]==2:
            ordersignal[i]=2
    df['ordersignal']=ordersignal

totalSignal(dfSPY, dfSPY.ADX_14)

In [30]:
#dfSPY[dfSPY.ordersignal!=0]

In [36]:
import numpy as np
def pointpos(x):
    if x['ordersignal']==1:
        return x['High']+2e-3
    elif x['ordersignal']==2:
        return x['Low']-2e-3
    else:
        return np.nan

dfSPY['pointpos'] = dfSPY.apply(lambda row: pointpos(row), axis=1)

In [37]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime

dfpl = dfSPY[1000:3200].copy()
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),
                go.Scatter(x=dfpl.index, y=dfpl.EMA, line=dict(color='orange', width=2), name="EMA")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=6, color="MediumPurple"),
                name="Signal")
fig.update_xaxes(rangeslider_visible=False)
fig.update_layout(autosize=False, width=600, height=600,margin=dict(l=50,r=50,b=100,t=100,pad=4), paper_bgcolor="white")
fig.show()

In [ ]:
dfpl = dfSPY[:].copy()
def SIGNAL():
    return dfpl.ordersignal

In [ ]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    initsize = 0.99
    mysize = initsize
    
    def init(self):
        super().init()
        self.signal = self.I(SIGNAL)

    def next(self):
        super().next()
        perc=0.02
        if len(self.trades)>0:
            if self.data.index[-1]-self.trades[-1].entry_time>=1000:
                self.trades[-1].close()
            if self.trades[-1].is_long and self.data.RSI[-1]>=75:
                self.trades[-1].close()
            elif self.trades[-1].is_short and self.data.RSI[-1]<=25:
                self.trades[-1].close()
        
        if self.signal==2 and len(self.trades)==0:  
            sl1 = min(self.data.Low[-1],self.data.Low[-2])*(1-perc)
            #tp1 = self.data.Close[-1]+(self.data.Close[-1] - sl1)*TPSLRatio
            #self.buy(sl=sl1, tp=tp1, size=self.mysize)
            self.buy(sl=sl1,size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=1000, margin=1/5, commission=.000)
stat = bt.run()
stat

In [ ]:
bt.plot()